In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [2]:
import pickle
import pandas as pd
import numpy as np

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [4]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-03.parquet')

In [6]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [13]:
# Calculating the standard deviation
# np.std(y_pred)

# Mean predicted duration
import statistics
statistics.mean(y_pred)


12.758556818790892

In [8]:
year=2022
month=3
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [9]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration,ride_id
0,1,2022-03-01 00:13:08,2022-03-01 00:24:35,1.0,2.40,1.0,N,90,209,2,...,3.0,0.5,0.00,0.0,0.3,13.80,2.5,0.00,11.450000,2022/03_0
1,1,2022-03-01 00:47:52,2022-03-01 01:00:08,1.0,2.20,1.0,N,148,234,2,...,3.0,0.5,0.00,0.0,0.3,14.30,2.5,0.00,12.266667,2022/03_1
2,2,2022-03-01 00:02:46,2022-03-01 00:46:43,1.0,19.78,2.0,N,132,249,1,...,0.0,0.5,11.06,0.0,0.3,67.61,2.5,1.25,43.950000,2022/03_2
3,2,2022-03-01 00:52:43,2022-03-01 01:03:40,2.0,2.94,1.0,N,211,66,1,...,0.5,0.5,4.44,0.0,0.3,19.24,2.5,0.00,10.950000,2022/03_3
4,2,2022-03-01 00:15:35,2022-03-01 00:34:13,1.0,8.57,1.0,N,138,197,1,...,0.5,0.5,5.51,0.0,0.3,33.06,0.0,1.25,18.633333,2022/03_4


In [12]:
df_result = df[['duration', 'ride_id']]

print(df_result['duration'].mean())

# df_result.to_parquet(
#     "resulting_dataframe.parquet",
#     engine="pyarrow",
#     compression=None,
#     index=False
# )

14.399294193379124
